1.Milestone 

1:[ Score: 40 points ] Input: Context and Dataset‣Process:

Step 1: Import the data [ 3 points ]

In [138]:
#import libraries 
import numpy as np
import pandas as pd
import re 
import nltk 
import keras
import tensorflow as tf
from tensorflow import keras
import os
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
import random
import json
import pickle
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
# nltk downloaded (run only once)
nltk.download('stopwords',quiet=True) # stopword library
nltk.download('wordnet', quiet=True) # wordnet library
nltk.download('words', quiet=True) # words library
nltk.download('punkt', quiet=True) # tokenize library

True

In [128]:
#Import data
project_path = 'C:/Users/OmSai/Documents/NLP Chatbot dataset/'
import pandas as pd
data = pd.read_csv( project_path + 'Data Set - industrial_safety_and_health_database_with_accidents_description.csv')

In [129]:
#Dropped Unnamed and unimportant column
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.drop(['Genre'], axis=1, inplace=True)

Step 2: Data cleansing [ 5 points ]

In [130]:
#Data overview
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Data                      425 non-null    object
 1   Countries                 425 non-null    object
 2   Local                     425 non-null    object
 3   Industry Sector           425 non-null    object
 4   Accident Level            425 non-null    object
 5   Potential Accident Level  425 non-null    object
 6   Employee or Third Party   425 non-null    object
 7   Critical Risk             425 non-null    object
 8   Description               425 non-null    object
dtypes: object(9)
memory usage: 30.0+ KB


In [141]:
#Rename column with meaningful label
data.rename(columns = {'Data':'Date & Timestamp'}, inplace = True)

In [142]:
#Data overview
data.describe()

,Date & Timestamp,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Employee or Third Party,Critical Risk,Description
count,418,418,418,418,418,418,418,418,418
unique,287,3,12,3,5,6,3,33,411
top,2017-02-08 00:00:00,Country_01,Local_03,Mining,I,IV,Third Party,Others,During the activity of chuteo of ore in hopper...
freq,6,248,89,237,309,141,185,229,2


In [143]:
#Null values checks
data.isna().any()

Date & Timestamp            False
Countries                   False
Local                       False
Industry Sector             False
Accident Level              False
Potential Accident Level    False
Employee or Third Party     False
Critical Risk               False
Description                 False
dtype: bool

In [144]:
#Null values checks
data.isnull().any()

Date & Timestamp            False
Countries                   False
Local                       False
Industry Sector             False
Accident Level              False
Potential Accident Level    False
Employee or Third Party     False
Critical Risk               False
Description                 False
dtype: bool

In [145]:
#Duplicate values checks
data.duplicated().any()

False

In [146]:
#Dropped Duplicate rows
data.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

In [154]:
# plt.bar(data['Industry Sector'],data['Accident Level'])
# plt.title('Accident severity w.r.t. Industry sector')
# plt.xlabel('Industry Sector')
# plt.ylabel('Accident Level')
# plt.show()

Step 3: Data preprocessing (NLP Preprocessing techniques) [ 7 points ]

In [148]:
#Preprocessing with NLP
def remove_punctuation(text):
    return re.sub('[^a-zA-Z]', ' ', str(text))

def lower_case(text):
    return text.lower()    

def remove_tags(text):    
    return re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)

def remove_special_chars_and_digits(text):
    return re.sub("(\\d|\\W)+"," ", text)

def stopword_lemma(text):
    token = nltk.word_tokenize(text)
    text_stop = [x for x in token if x not in set(stopwords.words('english'))]
    lemmatizer = WordNetLemmatizer()
    text_lemma = [lemmatizer.lemmatize(word) for word in text_stop]
    text_lemma = ' '.join(text_lemma)
    return text_lemma


def normalize_text(text: str) -> str:
    text = remove_punctuation(text)
    text = lower_case(text)
    text = remove_tags(text)
    text = remove_special_chars_and_digits(text)
    text = stopword_lemma(text)
    
    return text

In [149]:
data['normalized_Description'] = data['Description'].apply(normalize_text)

In [150]:
data

,Date & Timestamp,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Employee or Third Party,Critical Risk,Description,normalized_Description
0,2016-01-01 00:00:00,Country_01,Local_01,Mining,I,IV,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...,removing drill rod jumbo maintenance superviso...
1,2016-01-02 00:00:00,Country_02,Local_02,Mining,I,IV,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...,activation sodium sulphide pump piping uncoupl...
2,2016-01-06 00:00:00,Country_01,Local_03,Mining,I,III,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...,sub station milpo located level collaborator e...
3,2016-01-08 00:00:00,Country_01,Local_04,Mining,I,I,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...,approximately nv cx ob personnel begin task un...
4,2016-01-10 00:00:00,Country_01,Local_04,Mining,IV,IV,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...,approximately circumstance mechanic anthony gr...
...,...,...,...,...,...,...,...,...,...,...
420,2017-07-04 00:00:00,Country_01,Local_04,Mining,I,III,Third Party,Others,"Being approximately 5:00 a.m. approximately, w...",approximately approximately lifting kelly hq t...
421,2017-07-04 00:00:00,Country_01,Local_03,Mining,I,II,Employee,Others,The collaborator moved from the infrastructure...,collaborator moved infrastructure office julio...
422,2017-07-05 00:00:00,Country_02,Local_09,Metals,I,II,Employee,Venomous Animals,During the environmental monitoring activity i...,environmental monitoring activity area employe...
423,2017-07-06 00:00:00,Country_02,Local_05,Metals,I,II,Employee,Cut,The Employee performed the activity of strippi...,employee performed activity stripping cathode ...


Step 4: Data preparation - Cleansed data in .xlsx or .csv file [ 5 points ]

In [123]:
#Defining TF-IDF
vectorizer_tfidf = TfidfVectorizer(stop_words = set(nltk.corpus.stopwords.words('english')))
tfidf = vectorizer_tfidf.fit_transform(data.normalized_Description)
tfidf.shape 

(418, 2774)

In [124]:
## Create Data Frame of tdidf scores
tfidf_df = pd.DataFrame(tfidf.toarray(),
             columns = vectorizer_tfidf.get_feature_names(),
             index = data.index)

In [125]:
# Calculate tfidf for all columns and list top 10
tfidf_df.mean().sort_values(ascending = False).head(10) 

employee    0.035995
hand        0.032129
causing     0.030099
right       0.028796
left        0.028697
operator    0.026563
activity    0.025298
injury      0.022282
time        0.021045
pipe        0.020533
dtype: float64

Step 5: Design train and test basic machine learning classifiers [ 10 Points ]

In [126]:
# Training the model

## Train_Test Split
X_train, X_test, y_train, y_test = train_test_split(data['normalized_Description'].values,data['Critical Risk'].values,test_size=0.15, random_state=42)
print('Training utterances: {}'.format(X_train.shape[0]))
print('Validation utterances: {}'.format(X_test.shape[0]))

## TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

## Feature Selection
from sklearn.feature_selection import SelectKBest, chi2
ch2 = SelectKBest(chi2, k=2537)
X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)

## Model Testing
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
model_mnb_m2 = MultinomialNB()
model_mnb_m2.fit(X_train, y_train)
pred = model_mnb_m2.predict(X_test)
print("Accuaracy Score :",accuracy_score(y_test, pred))

Training utterances: 355
Validation utterances: 63
Accuaracy Score : 0.5873015873015873


Step 6: Interim report [ 10 points ].